In [3]:
import pandas as pd


data = pd.read_csv('../data/processed/clean_dataset_.csv')
components = pd.read_csv('../data/processed/principalComponents_.csv')

X = components
y = data['Fire Alarm']

